In [101]:
import numpy as np
from scipy.optimize import minimize, approx_fprime

# Fake dataset
np.random.seed(42)
X = np.random.randn(10_000, 2)
beta_true = np.array([1.0, -2.0])
p = 1 / (1 + np.exp(-(X @ beta_true)))
y = np.random.binomial(1, p)

# BCE loss function (negative log-likelihood)
def bce_loss(beta):
    z = X @ beta
    p_hat = 1 / (1 + np.exp(-z))
    eps = 1e-9
    return -np.sum(y * np.log(p_hat + eps) + (1 - y) * np.log(1 - p_hat + eps))

# Fit the model
res = minimize(bce_loss, x0=np.zeros(X.shape[1]), method="BFGS")
beta_hat = res.x
print("beta_hat:", beta_hat)


beta_hat: [ 1.06445722 -2.07456039]


In [102]:
def numerical_hessian(fun, x0, epsilon=1e-6):
    n = len(x0)
    H = np.zeros((n, n))
    for i in range(n):
        x_i_up = x0.copy()
        x_i_down = x0.copy()
        x_i_up[i] += epsilon
        x_i_down[i] -= epsilon

        grad_i_up = approx_fprime(x_i_up, fun, epsilon)
        grad_i_down = approx_fprime(x_i_down, fun, epsilon)

        H[:, i] = (grad_i_up - grad_i_down) / (2 * epsilon)
    return H


In [103]:
def hessian_logistic(beta):
    p_hat = 1 / (1 + np.exp(-(X @ beta)))
    w = p_hat * (1 - p_hat)                # N vector
    X_weighted = X * w[:, None]            # N x k
    return X.T @ X_weighted                 # k x k


In [108]:
# Hessian at beta_hat
H = numerical_hessian(bce_loss, beta_hat)

# Covariance matrix = inverse Hessian
cov_matrix = np.linalg.inv(H)
se = np.sqrt(np.diag(cov_matrix))

# 95% CI
z = 1.96
ci_lower = beta_hat - z * se
ci_upper = beta_hat + z * se

print("Standard errors:", se)
print("95% CI lower:", ci_lower)
print("95% CI upper:", ci_upper)


Standard errors: [0.03181739 0.04345654]
95% CI lower: [ 1.00209513 -2.1597352 ]
95% CI upper: [ 1.12681931 -1.98938557]


In [23]:
def se_ci_from_hessian(H, beta_hat, z=1.96):
    cov_matrix = np.linalg.inv(H)
    se = np.sqrt(np.diag(cov_matrix))
    ci_lower = beta_hat - z * se
    ci_upper = beta_hat + z * se
    return se, ci_lower, ci_upper

In [35]:
H = hessian_logistic(beta_hat)
cov_matrix = np.linalg.inv(H)
se = np.sqrt(np.diag(cov_matrix))

# 95% CI
z = 1.96
ci_lower = beta_hat - z * se
ci_upper = beta_hat + z * se

print("Standard errors:", se)
print("95% CI lower:", ci_lower)
print("95% CI upper:", ci_upper)


Standard errors: [0.03180573 0.04342601]
95% CI lower: [ 1.00211799 -2.15967536]
95% CI upper: [ 1.12679644 -1.98944541]


## Multiplicative Model: MSE and CE Heissan

In [1]:
from quantbullet.linear_product_model._test_utils import generate_linear_product_example_data, plot_predictions_by_bins
from quantbullet.linear_product_model import LinearProductRegressorScipy

In [2]:
import numpy as np
from scipy.optimize import approx_fprime

df, train_df, feature_groups = generate_linear_product_example_data( n_samples=100_000 )
lpr_scipy = LinearProductRegressorScipy()

In [3]:
lpr_scipy.fit(train_df, df['y'], feature_groups=feature_groups)

In [50]:
# df['clf_scipy_pred'] = lpr_scipy.predict(train_df)
# plot_predictions_by_bins(df, true_col='y', pred_col='clf_scipy_pred')

In [4]:
lpr_scipy.predict(train_df)

array([10.97399592, 14.2207433 , 10.03660043, ..., 10.34600834,
       10.4776577 , 11.09124633])

In [5]:
lpr_scipy.leave_out_feature_group_predict('x1', train_df)

array([12.70939763, 12.64263059, 10.91235164, ..., 11.6268746 ,
       12.39166416, 12.88042641])

In [6]:
lpr_scipy.leave_out_feature_group_predict('x2', train_df)

array([0.86345524, 1.12482471, 0.91974679, ..., 0.88983572, 0.84554081,
       0.8610931 ])

In [11]:
12.70939763 * 0.86345524

10.97399598086708

In [69]:
import numpy as np
from scipy.optimize import approx_fprime

# Your trained parameters
params_dict = lpr_scipy.coef_dict
y = df['y'].values

# Build betas and X_blocks from DataFrame
feature_blocks = list(params_dict.keys())
X_blocks = {}
beta_vec = []
block_slices = {}
start = 0
for block in feature_blocks:
    features = list(params_dict[block].keys())
    X_blocks[block] = train_df[features].values  # each feature is a column in df
    b = np.array([params_dict[block][f] for f in features])
    beta_vec.extend(b)
    block_slices[block] = slice(start, start + len(features))
    start += len(features)
beta_vec = np.array(beta_vec)


In [70]:
def hessian_mse_exact(beta, X_blocks, block_slices, y):
    # Build f_vals for each block
    betas = {g: beta[sl] for g, sl in block_slices.items()}
    f_vals = {g: X_blocks[g] @ betas[g] for g in X_blocks}
    yhat = np.prod(list(f_vals.values()), axis=0)

    # Precompute Q and R
    Q = {}
    for g in f_vals:
        prod = np.ones(len(y))
        for h in f_vals:
            if h != g:
                prod *= f_vals[h]
        Q[g] = prod

    # Build Hessian
    p = len(beta)
    H = np.zeros((p, p))
    for g in f_vals:
        Xg = X_blocks[g]
        for gp in f_vals:
            Xgp = X_blocks[gp]
            idx_g = np.arange(block_slices[g].start, block_slices[g].stop)
            idx_gp = np.arange(block_slices[gp].start, block_slices[gp].stop)
            if g == gp:
                H[np.ix_(idx_g, idx_gp)] = Xg.T @ ((Q[g]**2)[:, None] * Xgp)
            else:
                R = np.ones(len(y))
                for h in f_vals:
                    if h != g and h != gp:
                        R *= f_vals[h]
                H[np.ix_(idx_g, idx_gp)] = Xg.T @ (((Q[g] * Q[gp]) + (yhat - y) * R)[:, None] * Xgp)
    return H


In [76]:
def mse_loss(beta):
    betas = {g: beta[sl] for g, sl in block_slices.items()}
    f_vals = {g: X_blocks[g] @ betas[g] for g in X_blocks}
    yhat = np.prod(list(f_vals.values()), axis=0)
    return 0.5 * np.sum((yhat - y)**2)

def numerical_hessian(fun, beta, eps=1e-5):
    n = len(beta)
    H = np.zeros((n, n))
    for i in range(n):
        beta_up = beta.copy()
        beta_dn = beta.copy()
        beta_up[i] += eps
        beta_dn[i] -= eps
        grad_up = approx_fprime(beta_up, fun, eps)
        grad_dn = approx_fprime(beta_dn, fun, eps)
        H[:, i] = (grad_up - grad_dn) / (2 * eps)
    return H


In [77]:
H_exact = hessian_mse_exact(beta_vec, X_blocks, block_slices, y)
H_num = numerical_hessian(mse_loss, beta_vec)

print("Max abs diff:", np.max(np.abs(H_exact - H_num)))


Max abs diff: 1105.0003138966858


In [78]:
beta_vec

array([-0.60135797, -0.29842903, -0.21136406, -0.13342004, -0.03549496,
        0.03733245,  0.12995652,  0.20979408,  0.29988011, 14.04058783,
       -0.70067447, -3.37625507,  0.16335006,  3.36230729,  0.31021311,
       -3.31703575, -0.69977451,  2.95046555])

In [79]:
se_ci_from_hessian(H_num, beta_vec)

C:\Users\yzhang\AppData\Local\Temp;\ipykernel_37896\949678268.py:3: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(np.diag(cov_matrix))


(array([       nan,        nan,        nan,        nan, 0.00292247,
        0.00287894,        nan,        nan,        nan,        nan,
        0.04358581,        nan, 0.03858   ,        nan, 0.03717844,
               nan, 0.0300415 ,        nan]),
 array([        nan,         nan,         nan,         nan, -0.041223  ,
         0.03168973,         nan,         nan,         nan,         nan,
        -0.78610266,         nan,  0.08773326,         nan,  0.23734336,
                nan, -0.75865585,         nan]),
 array([        nan,         nan,         nan,         nan, -0.02976692,
         0.04297517,         nan,         nan,         nan,         nan,
        -0.61524627,         nan,  0.23896685,         nan,  0.38308286,
                nan, -0.64089317,         nan]))

In [80]:
se_ci_from_hessian(H_exact, beta_vec)

(array([0.11136736, 0.05516918, 0.03910284, 0.02480464, 0.00727972,
        0.00757913, 0.02416893, 0.03881382, 0.0554366 , 2.60619136,
        0.13853919, 0.62372872, 0.04930725, 0.62108722, 0.06911731,
        0.61276032, 0.13491562, 0.54616928]),
 array([-0.819638  , -0.40656063, -0.28800563, -0.18203712, -0.04976321,
         0.02247736,  0.08258542,  0.13371899,  0.19122437,  8.93245276,
        -0.97221127, -4.59876336,  0.06670785,  2.14497634,  0.17474318,
        -4.51804597, -0.96420912,  1.87997375]),
 array([-0.38307794, -0.19029743, -0.1347225 , -0.08480295, -0.02122671,
         0.05218753,  0.17732763,  0.28586917,  0.40853586, 19.1487229 ,
        -0.42913766, -2.15374679,  0.25999227,  4.57963824,  0.44568304,
        -2.11602552, -0.43533991,  4.02095734]))